In [1]:
import cv2
import numpy as np
import time


In [2]:
# Load YOLOv4 model and class labels
net = cv2.dnn.readNet('yolov4.weights', 'yolov4.cfg')
with open('coco.names', 'r') as f:
    classes = f.read().strip().split('\n')


In [3]:
# Initialize video capture (0 for the first camera or 'path_to_your_video.mp4' for a video file)
cap = cv2.VideoCapture('bombay traffic.mp4')

# Check if the video capture is successful
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()


In [6]:
# Function to process each frame
def process_frame(frame):
    height, width = frame.shape[:2]

    # Create a blob from the frame
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Get the output layer names
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    # Perform forward pass
    detections = net.forward(output_layers)

    # Initialize lists for detected boxes, confidences, and class IDs
    boxes = []
    confidences = []
    class_ids = []

    # Process each detection
    for output in detections:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # Filter out weak predictions
            if confidence > 0.5:
                # Get bounding box coordinates
                box = detection[0:4] * np.array([width, height, width, height])
                (centerX, centerY, w, h) = box.astype("int")
                x = int(centerX - (w / 2))
                y = int(centerY - (h / 2))

                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maxima suppression to suppress weak and overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw bounding boxes and class labels
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            color = (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
            cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return frame


In [ ]:
# Process video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process the frame
    processed_frame = process_frame(frame)

    # Display the processed frame
    cv2.imshow('YOLOv4 Object Detection', processed_frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import time

# Load YOLOv4 model and class labels
net = cv2.dnn.readNet('yolov4.weights', 'yolov4.cfg')
with open('coco.names', 'r') as f:
    classes = f.read().strip().split('\n')

# Initialize video capture (use 'traffic_video.mp4' as an example video file)
cap = cv2.VideoCapture('bombay traffic.mp4')

# Check if the video capture is successful
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter('output_video.avi', fourcc, fps, (width, height))

# Function to process each frame
def process_frame(frame):
    # Your existing processing code here...

# Process video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process the frame
    processed_frame = process_frame(frame)

    # Write the processed frame to the output video
    output_video.write(processed_frame)

    # Display the processed frame (optional)
    cv2.imshow('YOLOv4 Object Detection', processed_frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and writer and close all OpenCV windows
cap.release()
output_video.release()
cv2.destroyAllWindows()
